In [ ]:
create schema services

In [ ]:
Create table services.positions (
    position_id serial primary key,
    title char(30) not null,
    salary int not null
)

In [ ]:
INSERT INTO services.positions (title, salary)
    VALUES ('prorap', 1000),
           ('usta 1', 650),
           ('usta 2', 650),
           ('usta 3', 650),
           ('yordamchi 1', 450),
           ('yordamchi 2', 450),
           ('yordamchi 3', 450),
           ('qorovul 1', 800),
           ('qorovul 2', 800),
           ('qorovul 3', 800),
           ('haydovchi 1', 500),
           ('haydovchi 2', 500),
           ('haydovchi 3', 500);

In [ ]:
Create table services.units (
    unit_id serial primary key,
    title char(30) not null
)

In [ ]:
INSERT INTO services.units (title)
    VALUES ('m2 (metr kvadrat)'),
           ('m3 (metr kub)');

In [ ]:
Create table services.services (
    service_id serial primary key,
    name char(30) not null,
    duration float not null,
    unit int not null,
    unit_id int references services.units(unit_id) not null,
    price float
)

In [ ]:
INSERT INTO services.services (name, duration, unit, unit_id, price)
VALUES ('uy qurish', 20, 50, 1, 0),
       ('yer qazish', 2, 12, 2, 0);

In [ ]:
Create table services.teams (
    team_id serial primary key,
    service_id int references services.services(service_id) not null,
    position_id int references services.positions(position_id) not null
)

In [ ]:
INSERT INTO services.teams (service_id, position_id)
    VALUES (1, 1),
           (1, 2),
           (1, 3),
           (1, 4),
           (2, 5),
           (2, 6),
           (2, 7),
           (2, 8);

Triggers

In [ ]:
Create or Replace function services.after_inserting()
    returns TRIGGER
    LANGUAGE plpgsql
    AS
    $$
BEGIN
    Update services.services set price = (
        Select sum(salary) from services.positions
            where position_id in (
                select position_id from services.teams
                    WHERE service_id=new.service_id))
                        where service_id=new.service_id;
    RETURN new;
END;
$$;

In [ ]:
CREATE or Replace TRIGGER new_team_member
    After Insert
    ON services.teams
    FOR EACH ROW
    EXECUTE PROCEDURE services.after_inserting();

In [ ]:
Create or Replace function services.before_deleting()
    returns TRIGGER
    LANGUAGE plpgsql
    AS
    $$
BEGIN
    Update services.services set price = price - (
        Select salary from services.positions
            where position_id = OLD.position_id)
        where service_id=OLD.service_id;

    RETURN OLD;
END
$$;

In [ ]:
CREATE or Replace TRIGGER remove_team_member
    Before Delete
    ON services.teams
    FOR EACH ROW
    EXECUTE PROCEDURE services.before_deleting();

In [ ]:
Create or Replace function before_updating()
returns TRIGGER
LANGUAGE plpgsql
AS
$$
BEGIN
    IF NEW.salary <> OLD.salary THEN
        if NEW.salary > OLD.salary THEN
            Update services.services set price = price + (new.salary - old.salary)
                where position_id in (
                    select position_id from services.teams
                        WHERE service_id=new.service_id);
        else
            Update services.services set price = price - (old.salary - new.salary)
                where position_id in (
                    select position_id from services.teams
                        WHERE service_id=new.service_id);
        END IF;
    END IF;

    RETURN NEW;
END
$$;

In [ ]:
CREATE or Replace TRIGGER before_updating
    BEFORE UPDATE
    ON services.positions
    FOR EACH ROW
    EXECUTE PROCEDURE before_updating();

In [ ]:
select price, array_to_json(array_agg(salary)) from services.positions
inner join services.teams using(position_id)
inner join services.services using(service_id)
group by price

In [ ]:
select price, salary from services.positions
inner join services.teams using(position_id)
inner join services.services using(service_id)

In [ ]:
salary ni insert qilganda
    price + old.salary
salary -> update
    if NEW.salary <> OLD.salary
        if NEW.salary > OLD.salary
            price + (new.salary - old.salary)
        else:
            price - (old.salary - new.salary)
salary -> delete
    price - old.salary

In [ ]:
delete from services.teams where position_id = 5

In [ ]:
update services.positions set salary = 2000 where position_id = 1